In [138]:
from numpy.core.numeric import roll
from solid.objects import circle, cube, cylinder, linear_extrude, offset, rotate, rotate_extrude, sphere, square, translate, import_scad, hull
from solid.utils import right, up, forward
from viewscad import Renderer


r = Renderer()
r.width = 600
r.height = 600

render = r.render


def spin(obj, amount=3):
    return sum((rotate((0, 0, a))(obj) for a in range(0, 360, 360//amount)))


def peristaltic_pump(d=60, tube=12.5, tube_wall=2.5, rolls=5, support=4, key_size=6):
    h = tube+2
    sphere_center_offset = (d/2-(h//3*2))
    out_ring = translate(((d/2-h), -h/2, 0))(square(size=h+1))\
        - translate((d/2-(h//2*3), -h/2, 0))(square(size=h+1))\
        - right(sphere_center_offset)(circle(d=tube, segments=100, ))\

    bullet = right(sphere_center_offset)(
        sphere(d=tube-tube_wall, segments=100, ) +
        cylinder(d=support, h=h, center=True, segments=100, )
    )
    bullets = spin(bullet, rolls)

    socket = translate((0, -h/2, -h/2))(cube((sphere_center_offset-(tube/4), h, h)))\
        - right(sphere_center_offset)(sphere(d=tube, segments=100, ))

    socket_roller = spin(socket, rolls)\
        - cylinder(d=key_size, segments=100, h=h, center=True)

    socket_roller *= cylinder(h=h, r=sphere_center_offset-tube/4, segments=100, center=True)

    inlet = rotate((0, 90, 0))(cylinder(d=tube, h=d, segments=100, ))
    inlets = forward(sphere_center_offset)(inlet) + \
        forward(-sphere_center_offset)(inlet)\

    return rotate_extrude(segments=100, )(out_ring) - inlets + bullets + socket_roller


render(peristaltic_pump(), outfile='./outfiles/peristaltic_pump.stl')
